In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as metricas
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

## Actividad sugerida

Abajo vas a ver tabajo con otro set de datos, que no está tan desbalanceado. Pero lo que puedes hacer es borrar algunos ejemplos con fraude, para desbalancear más el set de datos y hacer que sea importante alguna acción correctiva. 

Realiza esa acción, y comprueba para distintos conjuntos de datos borrados (digamos, borrando el 90% de los fraudes, y el 99% de los fraudes) si SMOTE logra un mejor rendimiento, o no.

In [2]:
datos = pd.read_csv('card_transdata.csv')

fraudes = datos[datos['fraud'] == 1.0]


# 90%
### Código que remueve el 90% de los datos clasificados como fraude
filas_a_remover90 = fraudes.sample(frac=0.9, random_state=42)
datos90 = datos.drop(filas_a_remover90.index)

### Código que remueve el 99% de los datos clasificados como fraude
filas_a_remover99 = fraudes.sample(frac=0.99, random_state=42)
datos99 = datos.drop(filas_a_remover99.index)


In [ ]:
def evaluar_modelo(datos):
    # Verificar si hay valores NaN en el conjunto de datos
    if datos.isnull().values.any():
        print("Hay valores NaN en el conjunto de datos. Procediendo a eliminarlos.")
        datos = datos.dropna()  

    X = datos.drop('fraud', axis=1)
    y = datos['fraud']
    
    # Dividir el conjunto en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Aplicar SMOTE
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
    # Entrenar los modelos
    modelo_smote = RandomForestClassifier(random_state=42)
    modelo_no_smote = RandomForestClassifier(random_state=42)

    modelo_smote.fit(X_train_res, y_train_res)
    modelo_no_smote.fit(X_train, y_train)
    
    # Realizar predicciones
    y_pred_smote = modelo_smote.predict(X_test)
    y_pred_no_smote = modelo_no_smote.predict(X_test)

    
    # Evaluar el rendimiento
    return accuracy_score(y_test, y_pred_smote), accuracy_score(y_test, y_pred_no_smote)

In [4]:
#La celda se demora 16 segundos, por favor tengan paciencia :(
rendimiento_smote90, rendimiento_no_smote90 = evaluar_modelo(datos90)

Hay valores NaN en el conjunto de datos. Procediendo a eliminarlos.


In [5]:
def conclusiones (rendimiento_smote, rentimiento_no_smote, porcentaje:int):
    print(f"El rendimiento (accuracy score) al eliminar el {porcentaje}% \n\tEl rendimiento sin ocupar SMOTE es {rentimiento_no_smote}\n\tEl rendimiento cuando ocupamos SMOTE {rendimiento_smote}")
    if rendimiento_smote > rendimiento_smote:
        print("Usar SMOTE es peor para el acurracy del modelo")
    elif rendimiento_smote == rendimiento_smote:
        print("Usar SMOTE no afecta ni para bien, ni para mal (o la diferencia es insignificante)")
    else: 
        print("Usar SMOTE mejora el acuracy del modelo")
    

In [6]:
conclusiones(rendimiento_smote90, rendimiento_no_smote90, 90)

El rendimiento (accuracy score) al eliminar el 90% 
	El rendimiento sin ocupar SMOTE es 0.999736931113533
	El rendimiento cuando ocupamos SMOTE 0.999473862227066
Usar SMOTE no afecta ni para bien, ni para mal (o la diferencia es insignificante)


In [7]:
## Repetimos el proceso para 99%

rendimiento_smote99, rendimiento_no_smote99 = evaluar_modelo(datos99)

Hay valores NaN en el conjunto de datos. Procediendo a eliminarlos.


In [8]:
conclusiones(rendimiento_smote99, rendimiento_no_smote99, 99)

El rendimiento (accuracy score) al eliminar el 99% 
	El rendimiento sin ocupar SMOTE es 0.9996588712428458
	El rendimiento cuando ocupamos SMOTE 0.9995072584618884
Usar SMOTE no afecta ni para bien, ni para mal (o la diferencia es insignificante)


En ambos casos el rendimiento no se ve afectado por el SMOTE, teorizo que es porque no están tan desbalanceados los datos. 